In [40]:
import json
from typing import List

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, StrOutputParser
import pandas as pd

In [41]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)

In [42]:
class Turn(BaseModel):
    role: str = Field("사람 이름")
    content: str = Field("대화 내용")
    
class Conversation(BaseModel):
    conversation: List[Turn] = Field("두 사람의 대화 내용")

In [43]:
parser = JsonOutputParser(pydantic_object=Conversation)

In [44]:
format_instruction = parser.get_format_instructions()

## Fewshot Example 만들기

In [45]:
#대화 내용을 성공적으로 만들기 위해 프롬프트 설정.
prompt_template = PromptTemplate.from_template(template="영어 단어'{input}'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}",
                                               partial_variables={"format_instruction": format_instruction})

In [46]:
#{"input": RunnablePassthrough()} 인풋을 그 상태 그대로 체인에 입력.
chain = {"input": RunnablePassthrough()}| prompt_template | model | parser

In [47]:
#단어 "request"를 활용한 실생활 대화 추출.
example_1 = chain.invoke("request")

In [48]:
example_1

{'conversation': [{'role': 'A', 'content': 'Can I make a request?'},
  {'role': 'B', 'content': 'Sure, what do you need?'},
  {'role': 'A', 'content': 'I request that you help me with my project.'},
  {'role': 'B', 'content': "Of course, I'd be happy to help."}]}

In [49]:
example_2 = chain.invoke("have")

In [50]:
example_2

{'conversation': [{'role': 'Aiden',
   'content': 'I have a meeting at 2 PM. Can we reschedule our lunch?'},
  {'role': 'Sophia',
   'content': 'Sure, I have some free time in the afternoon. How about 4 PM?'}]}

## Fewshot Example 적용해서 샘플링하기

In [51]:
#fewshot example을 적용하기 위해 대화 예제를 json형태로 파싱하기 위한 변수 선언.
example_1_json = json.dumps(example_1)
example_2_json = json.dumps(example_2)

In [52]:
example_1_json

'{"conversation": [{"role": "A", "content": "Can I make a request?"}, {"role": "B", "content": "Sure, what do you need?"}, {"role": "A", "content": "I request that you help me with my project."}, {"role": "B", "content": "Of course, I\'d be happy to help."}]}'

In [53]:
example_2_json

'{"conversation": [{"role": "Aiden", "content": "I have a meeting at 2 PM. Can we reschedule our lunch?"}, {"role": "Sophia", "content": "Sure, I have some free time in the afternoon. How about 4 PM?"}]}'

In [54]:
type(example_1_json)

str

In [55]:
msgs = [
    ("human", "영어 단어'request'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}"),
    ("ai", "{example_1_json}"),
    ("human", "영어 단어'have'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}"),
    ("ai", "{example_2_json}"),
    ("human", "영어 단어'{input}'에 대한 실생화 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}") #{input}은 외부에서 입력된 값을 invoke 해서 가져오게 됨.
]

In [56]:
chat_prompt_template = ChatPromptTemplate.from_messages(msgs)

In [57]:
#json 형식으로 parsing 할때 중괄호가 들어가면 랭체인에서 인식에 오류가 생기므로, 랭체인에서 json형식을 그대로 parsing할 수 있도록 lambda로 설정.
fewshot_chain = {"input": RunnablePassthrough(),
                 "format_instruction": lambda _:format_instruction,
                 "example_1_json": lambda _:example_1_json, 
                 "example_2_json": lambda _:example_2_json}  | chat_prompt_template | model | parser

In [58]:
out = fewshot_chain.invoke("multiple")
out

{'conversation': [{'role': 'Liam',
   'content': 'I have multiple tasks to complete by the end of the day.'},
  {'role': 'Ella',
   'content': "I can help you with some of them. Let's divide and conquer."},
  {'role': 'Liam',
   'content': 'That would be great. Thank you for offering your assistance.'}]}

## Load Excel and Save

In [59]:
df = pd.read_excel("./words.xlsx")

In [33]:
df = df.drop(columns=["usage"]) #미리 만들어진 예제 삭제.

KeyError: "['usage'] not found in axis"

In [35]:
fewshot_chain = {"input": RunnablePassthrough(),
                 "format_instruction": lambda _:format_instruction,
                 "example_1_json": lambda _:example_1_json, 
                 "example_2_json": lambda _:example_2_json}  | chat_prompt_template | model | StrOutputParser() # json그대로 엑셀파일에 저장해야되서 parser대신 StrOutputParser()로 처리.

In [ ]:
#엑셀의 각 행의 결과를 처리하는 코드.

In [36]:
df["usage"] = df.apply(lambda row: fewshot_chain.invoke(row["word"]), axis=1)

In [37]:
df.to_excel("./words.xlsx", index=False)

In [39]:
df

,word,imoj,meaning,usage
0,effort,💪,effort은 노력이나 수고를 의미합니다. 그것은 어떤 일을 이루기 위해 투자한 에...,"{""conversation"": [{""role"": ""Amanda"", ""content""..."
1,excitement,🎉,excitement은 흥분이나 설렘을 의미합니다. 어떤 일이나 사건이 특히 기쁘거나...,"{\n\t""conversation"": [{\n\t\t\t""role"": ""Emma"",..."
2,love,❤️,"love은 강한 애정이나 연애, 친밀한 관계를 의미합니다. 또한 사랑은 다른 사람에...","{""conversation"": [{""role"": ""Alice"", ""content"":..."
3,friendship,👫,friendship은 친구 사이의 관계를 의미합니다. 이는 서로 신뢰하고 존중하는 ...,"{""conversation"": [{""role"": ""Alice"", ""content"":..."
4,excited,🎉,"excited는 흥분한, 들뜬 감정이나 기분을 나타냅니다. 이 단어는 기대했던 일이...","{""conversation"": [{""role"": ""Emily"", ""content"":..."
5,success,🎉,success는 성공을 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 얻었을 때...,"{""conversation"": [{""role"": ""Alice"", ""content"":..."
6,success,NaN,success는 성공이나 성취를 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 ...,"{""conversation"": [{""role"": ""Alice"", ""content"":..."
7,happy,😃,happy는 기쁨이나 만족감을 의미합니다. 그것은 긍정적인 감정이나 좋은 상태를 표...,"{""conversation"": [{""role"": ""Alice"", ""content"":..."
8,success,🎉,success은 성공이나 성취를 의미합니다. 어떤 목표나 업적을 달성하거나 원하는 ...,"{""conversation"": [{""role"": ""Alice"", ""content"":..."
9,laughter,😂,"laughter는 웃음을 의미합니다. 그것은 기쁨, 유쾌함, 웃기거나 재미있는 것으...","{""conversation"": [{""role"": ""Alice"", ""content"":..."
